各階段設計思路
最後更新：2022/06/18

階段零、決定選股的標準。

在開始進行系統開發之前，我們應該要先來思考看看，如果是一般情況下，我們會怎麼挑選股票呢？

在股市界中，新手最常聽到也最容易操作的選股法則是:KD指標20買；KD指標80賣。
但這真的是比較好的法則嗎？
我想或許不是。
若我們以歷史數據回測(參考此篇文章：https://havocfuture.tw/blog/0050-kd-backtesting )。
我們會發現雖然勝率不錯，但可能會出現明明市場處於大多頭風向，KD指標卻出現賣出訊號，同時KD指標有可能在整個大多頭期間都不會再有買入訊號的出現，因而錯過續漲的獲利空間！

那既然KD指標20買；KD指標80賣的法則可能會錯過續漲空間，那我們還有其他選股法則嗎？

在《一條線搞定當沖、波段、存股！：飆股達人陳弘月賺50％，勝率8成的投資心法》這本書中，作者提出其自創的28均線選股法則。
意即在股價向上突破28K值均線時，進場買入；反之，向下突破28K值均線，出場賣出。同時合併成交量來判斷是否適合買入/賣出。
若按照書內作者的這套操作方法，或許能夠在應該賣時賣出，且此選股法則也有能力看準時機追加進場。於是，我們先暫定以這套系統來實際操作看看

階段一、分析與設計系統

在階段零中，我們已經決定要使用28均線法則來做為篩選準則。
接下來我們需要分流程來設計系統，並逐一實現。

我大致將系統流程分為以下部分：
初始設定 >> 獲取清單(目前整合至初始設定) >> 查詢個股 >> 計算指標 >> 準則判斷 >> 匯入清單
                                          ∧                                    ∨
                                          ∧                                    ∨
                                          <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

初始設定：包含基本套件匯入、函式定義及各代數的預設初始給值
獲取清單：從證交所取回最新清單
查詢個股：包括價格及成交量
計算指標：計算各項判斷要有的各項指標
準則判斷：透過計算的指標進行判斷
匯入清單：符合判斷準則就匯入其對應的清單，後續將視情況匯出

階段二、設計系統(初始設定)

接下來在系統的運行中，有些部分是需要先定義的。
例如函式、數值及空串列。但如果每次都是要用到的時候才給定義的話，版面或許會有些凌亂。因此我選擇將它納入進最前面的部分，也就是程式運行篩選迴圈前，就先弄完定義，這樣空間應該會整潔不少。
下列我們看系統初始設定的部分。

首先，我們在運行時，會使用到一些原本Python不會預設附帶的套件，因此我們需要利用pip指令來安裝。
以Google Colab為例，我們會需要額外安裝twstock與yfinance。
至於一般有預設的套件則是直接import即可，不需重複安裝。
由於套件名稱有點冗長，於是我也另外給定對應的代號。

In [ ]:
!pip install twstock
!pip install yfinance
import twstock as t 
import pandas as p 
from bs4 import BeautifulSoup 
import requests 
import time as ti
import os as o

接下來是定義函式的部分。
這個系統中我們運用到兩個函式，一個是網頁查詢使用，另一個是Line Notify通知。
由於本身對於api串接的熟悉度不是很好，因此這邊參考他人的程式代碼:(https://jeffwen0105.com/python_linenotify/)

In [ ]:
def web(url):#定義網頁查詢函式
    source = requests.get(url, headers={'Connection':'close'}) #連線到指定的網站
    #因為股票要篩選好幾千個，若我們一直開啟新網頁而不關掉已使用完畢的頁面，電腦資源會被浪費
    soup = BeautifulSoup(source.content, 'lxml', from_encoding='utf-8') #讀取這個網頁的內容
    return soup #最終結果:回傳網頁內容

def post_data(message, token):#Line notify傳訊息
    try:
        url = "https://notify-api.line.me/api/notify"#api網址
        headers = {
            'Authorization': f'Bearer {token}'
        }
        payload = {
            'message': message
        }
        response = requests.request(
            "POST",
            url,
            headers=headers,
            data=payload
        )
        if response.status_code == 200:
            print(f"Success -> {response.text}")
    except Exception as _:
        print(_)#狀態

接下來是取得最新股票資料。
在這邊我們利用pandas套件去讀取網頁內容，因上市及上櫃股為不統網頁因此這邊要取得兩次資料。
我們先來看如果直接至表會變成怎樣！

In [2]:
import pandas as p 
data = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=2', encoding='big5hkscs', header=0)[0]
print(data)

           有價證券代號及名稱 國際證券辨識號碼(ISIN Code)         上市日 市場別   產業別 CFICode   備註
0                 股票                  股票          股票  股票    股票      股票   股票
1            1101　台泥        TW0001101004  1962/02/09  上市  水泥工業  ESVUFR  NaN
2            1102　亞泥        TW0001102002  1962/06/08  上市  水泥工業  ESVUFR  NaN
3            1103　嘉泥        TW0001103000  1969/11/14  上市  水泥工業  ESVUFR  NaN
4            1104　環泥        TW0001104008  1971/02/01  上市  水泥工業  ESVUFR  NaN
...              ...                 ...         ...  ..   ...     ...  ...
30488  01003T　兆豐新光R1        TW00001003T4  2005/12/26  上市   NaN  CBCIXU  NaN
30489  01004T　土銀富邦R2        TW00001004T2  2006/04/13  上市   NaN  CBCIXU  NaN
30490  01007T　兆豐國泰R2        TW00001007T5  2006/10/13  上市   NaN  CBCIXU  NaN
30491  01009T　王道圓滿R1        TW00001009T1  2018/06/21  上市   NaN  CBCIXU  NaN
30492  01010T　京城樂富R1        TW00001010T9  2018/12/05  上市   NaN  CBCIXU  NaN

[30493 rows x 7 columns]


因為我們只需要 "有價證券代號及名稱" 的這部分資訊，因此我們單獨把這部分拉來製表後，再替代掉原先的表格。

In [3]:
import pandas as p 
data = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=2', encoding='big5hkscs', header=0)[0]
data = p.DataFrame(data["有價證券代號及名稱"])
data = data["有價證券代號及名稱"].astype(str)
print(data)

0                   股票
1              1101　台泥
2              1102　亞泥
3              1103　嘉泥
4              1104　環泥
             ...      
30488    01003T　兆豐新光R1
30489    01004T　土銀富邦R2
30490    01007T　兆豐國泰R2
30491    01009T　王道圓滿R1
30492    01010T　京城樂富R1
Name: 有價證券代號及名稱, Length: 30493, dtype: object


接下來我們只需要重複上市股的取得動作，但把網址改成上櫃股的網址，這樣我們就完成取得上市股跟上櫃股的最新清單步驟了！

In [ ]:
#####取得上市股#####
data = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=2', encoding='big5hkscs', header=0)[0]
data = p.DataFrame(data["有價證券代號及名稱"])
data = data["有價證券代號及名稱"].astype(str)

#####取得上櫃股#####
data2 = p.read_html('http://isin.twse.com.tw/isin/C_public.jsp?strMode=4', encoding='big5hkscs', header=0)[0]
data2 = p.DataFrame(data2["有價證券代號及名稱"])
data2 = data2["有價證券代號及名稱"].astype(str)

In [ ]:
#####創建空串列#####
stocklist=[]#篩選股票清單
stock28list = []#符合條件清單
amu28list = []#符合精選條件清單
ulti28list = []#符合當沖精選條件清單

#####代數預設指定#####
d1s = 1 #上市股開始index碼
d1e = 968 #上市股結束index碼
d2s = 7408 #上櫃股開始index碼
d2e = 8205 #上櫃股結束index碼

fast_start = int(input("冷啟動請輸入0|快啟動請輸入1:"+"\n上市股始/終:"+str(d1s)+"/"+str(d1e)+"\n上櫃股始/終:"+str(d2s)+"/"+str(d2e)+"\n"))
#####檢查上市股代數變動#####
check_data_index = int(input("請問上市股index是否有改變?(0無1有)"+"\n目前預設起始/終止值為:"+str(d1s)+"/"+str(d1e)))

if check_data_index == 0 and fast_start == 0:
    d1s = 1 
elif check_data_index == 1 and fast_start == 0:
  d1s = int(input("請輸入上市股起始index:"))
  d1e = int(input("請輸入上市股終止index:"))

#####檢查上櫃股代數變動#####
check_data2_index = int(input("請問上櫃股index是否有改變?(0無1有)"+"\n目前預設起始/終止值為:"+str(d2s)+"/"+str(d2e)))

if check_data2_index == 0 and fast_start == 0:
    d2s = 7408
elif check_data2_index == 1 and fast_start == 0:
  d2s = int(input("請輸入上櫃股起始index:"))
  d2e = int(input("請輸入上櫃股終止index:"))